# Train an IntegratedML model on Marketing Campaign Dataset
## Using InterSystems IRIS DB-API Driver
This Notebook demonstrates:
- Using the `intersystems-irispython` DB-API driver to connect to InterSystems IRIS
- Creating views to segment data into training and test sets
- Defining and training an IntegratedML model to predict marketing campaign responses
- Comparing the resulting model's predictions to data in the test set (that the model was not trained on)
- Using the IntegratedML "VALIDATE MODEL" command to calculate accuracy metrics on the test set data

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### 1. Set environment variables, if necessary

### 2. Get connection and cursor

In [ ]:
# InterSystems IRIS DB-API Driver connection
# Using the official intersystems-irispython package from PyPI
# https://pypi.org/project/intersystems-irispython/
import iris

# Connection configuration
connection_string = "irisimlsvr:1972/USER"
username = "SUPERUSER"
password = "SYS"

In [ ]:
# Establish connection using DB-API
conn = iris.connect(connection_string, username, password)
curs = conn.cursor()

### 3. specify the source data table

In [ ]:
dataTable = 'Marketing.Campaign'
trainingTable = "Marketing.CampaignTrainingSmall"
predictTable = "Marketing.CampaignPredictSmall"

### 4. Execute a query and display results in Pandas DataFrame

In [ ]:
import pandas as pd

df = pd.read_sql("select TOP 20 * from %s" % dataTable, conn)
display(df)

In [ ]:
# Show number rows
df1 = pd.read_sql("SELECT COUNT(*) FROM %s" % dataTable, conn)
display(df1)

### Cleaning before retrying

In [ ]:
# Before executing the notebook again, drop model, tables, views,... previously created
#curs.execute("DROP VIEW %s" % trainingTable)
#curs.execute("DROP VIEW %s" % predictTable)
#curs.execute("DROP MODEL CampaignModel")

### 5. Make some views to split training and testing datasets

In [ ]:
# Small training set view
curs.execute("CREATE VIEW %s AS SELECT * FROM %s WHERE ID<3001" % (trainingTable,dataTable))
# Small prediction set
curs.execute("CREATE VIEW %s AS SELECT * FROM %s WHERE ID>47000" % (predictTable,dataTable))

### 6. Create and Train an IntegratedML Model using default settings
IntegratedML only needs a model name, the name of the column that is the target column to predict, and a table (or SELECT query to specify input columns.

In [ ]:
curs.execute("CREATE MODEL CampaignModel PREDICTING (RESPONSE) FROM %s" % trainingTable)

Now that the model is defined, you can TRAIN it, which invokes the AutoML machine learning procedure.

In [ ]:
curs.execute("TRAIN MODEL CampaignModel")

Once that finishes, you can see some information about the model in the "ML_TRAINED_MODELS" table.

In [ ]:
df3 = pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.ML_TRAINED_MODELS", conn)
display(df3)

### 7. Compare model output to data it has not seen yet
Now you can use SQL to SELECT data from another table, run the IntegratedML model on this new data, and see how well the predictions match the data!

In [ ]:
df4 = pd.read_sql("SELECT PREDICT(CampaignModel) AS PredictedResponse, \
                  response AS ActualResponse FROM %s" % predictTable, conn)
display(df4)

### 8. VALIDATE MODEL command calculates accuracy metrics
You can certainly take that output above and calculate the accuracy using a standard formula, but IntegratedML has a built-in function to do that!

Each time you run the command "VALIDATE MODEL..." it generates a set of metrics calculated on the data passed into the query. Since this table can be a bit difficult to read in its raw form we use a simple "pivot" call to arrange the data.

In [ ]:
# Run VALIDATE MODEL to generate accuracy metrics
curs.execute("VALIDATE MODEL CampaignModel FROM %s" % predictTable)

# Get the most recent validation run for this model
df5 = pd.read_sql("""
    SELECT * FROM INFORMATION_SCHEMA.ML_VALIDATION_METRICS 
    WHERE MODEL_NAME = 'CampaignModel'
    ORDER BY VALIDATION_RUN_NAME DESC
""", conn)

# Get only the latest validation run and remove any duplicate metrics
latest_run = df5['VALIDATION_RUN_NAME'].iloc[0]
df_latest = df5[df5['VALIDATION_RUN_NAME'] == latest_run].drop_duplicates(
    subset=['VALIDATION_RUN_NAME', 'METRIC_NAME'], keep='first'
)

# Pivot to display metrics in a readable format
df6 = df_latest.pivot(index='VALIDATION_RUN_NAME', columns='METRIC_NAME', values='METRIC_VALUE')
display(df6)